Loading Data:

In [494]:
import pandas as pd

# Load the Excel file into a DataFrame
data = pd.read_excel("Survey_ICS487_Project.xlsx")


Data Inspection:

In [495]:
# Check the first few rows
print(data.head())

# Get an overview of the data
print(data.info())

# Check for missing values
print(data.isnull().sum())


     Name  Salary (SAR)  Age    State     Sex  Monthly Debt (SAR)  \
0   sarah         16409   52   Single  Female              501.91   
1  Zainab          2980   24  Married  Female              457.79   
2  Mariam          1898   35  Married    Male              200.87   
3   Yazan          1679   59  Married  Female              296.33   
4   musab          4278   25   Single    Male             1039.79   

   Elementary Expenses (SAR)        Goal  Number of Children  \
0                     829.22  Investment                   0   
1                    8852.17  Investment                   2   
2                   11773.95         NaN                   4   
3                   12087.75     Savings                   1   
4                    1193.65         NaN                   0   

  Employment_Status Nationality Budgeting_Rule Financial_Comfortability  \
0          Employed       Saudi       70/20/10                   ['no']   
1           Student       Saudi       50/30/20    

Clean and Preprocess:

In [496]:
# Standardize column names
data.columns = [col.strip().lower().replace(" ", "_") for col in data.columns]
# Check for duplicates and remove them
data = data.drop_duplicates()

# Handle missing values (drop rows or fill them as needed)
data = data.dropna()  # Drop rows with missing values
# Alternatively, fill missing values (example: fill salary with median)
data['salary_(sar)'] = data['salary_(sar)'].fillna(data['salary_(sar)'].median())

# Ensure numeric columns are properly formatted
numeric_columns = ['salary_(sar)', 'age', 'monthly_debt_(sar)', 'elementary_expenses_(sar)']
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Validate ranges for numeric columns
data = data[(data['age'] >= 18) & (data['age'] <= 100)]  # Age should be between 0 and 100
data = data[data['salary_(sar)'] >= 0]  # Salary should not be negative

# Filter out rows where the conditions are met
filtered_data = data[
    (data['nationality'].str.lower() == 'saudi') ]  # Keep only rows where nationality is 'saudi'
filtered_data = filtered_data[
    (filtered_data['financial_comfortability'].str.lower() != 'no') ]  # Exclude rows where financial_comfortability is 'no'
filtered_data = filtered_data[
    (data['goal_progress'].str.lower() != 'no')  # Exclude rows where goal_progress is 'no'
]

# Drop the irrelevant columns
filtered_data = filtered_data.drop(columns=['nationality', 'financial_comfortability', 'goal_progress'])

# Store the filtered data back if needed
data = filtered_data
print(data.tail())



          name  salary_(sar)  age    state     sex  monthly_debt_(sar)  \
1165      Huda          5985   19  Married  Female              857.55   
1168     Rawan          2157   21   Single  Female              487.50   
1170  Mohammed          3255   22   Single    Male              741.37   
1171   Fatimah          2626   40   Single    Male              113.82   
1173      Asma          1117   19   Single  Female              138.70   

      elementary_expenses_(sar)        goal  number_of_children  \
1165                   13473.32  Investment                   0   
1168                    2729.23     Savings                   0   
1170                    3975.68  Investment                   0   
1171                    1221.78     Savings                   0   
1173                    1318.05     Savings                   0   

     employment_status budgeting_rule  
1165          Employed       50/30/20  
1168           Student       50/30/20  
1170           Student       50/

/var/folders/yc/7vmb_7z55z3d92njbdlxxr3m0000gp/T/ipykernel_61078/1195411881.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[


Removing outliers:

In [497]:
import numpy as np

def remove_outliers_iqr(df, column):
    """
    Removes outliers from a column using the IQR method.
    """
    Q1 = df[column].quantile(0.25)  # First quartile
    Q3 = df[column].quantile(0.75)  # Third quartile
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply to numeric columns with potential outliers
numeric_columns = ['salary_(sar)', 'age', 'monthly_debt_(sar)', 'elementary_expenses_(sar)']
for col in numeric_columns:
    data = remove_outliers_iqr(data, col)


# Define age bins and labels
age_bins = [0, 24, 34, 44, 54, float('inf')]
age_labels = ['18-24', '25-34', '35-44', '45-54', '55+']

# Create a new column for age groups
data['age_group'] = pd.cut(data['age'], bins=age_bins, labels=age_labels, right=True)

data = data.drop(columns=['age'])


Encoding Categorical Variables:

In [498]:
from sklearn.preprocessing import LabelEncoder

# One-hot encode categorical columns
categorical_columns = ['state', 'sex', 'goal', 'employment_status']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Encode the age_group using LabelEncoder
label_encoder = LabelEncoder()
data['age_group_encoded'] = label_encoder.fit_transform(data['age_group'])

# Display the encoding
print("Age Group Encoding:")
for label, encoded in zip(label_encoder.classes_, range(len(label_encoder.classes_))):
    print(f"{label}: {encoded}")

data = data.drop(columns=['age_group'])


Age Group Encoding:
18-24: 0
25-34: 1
35-44: 2
45-54: 3
55+: 4


Feature Engineering:

In [499]:
# Create new features
data['debt_to_income_ratio'] = data['monthly_debt_(sar)'] / data['salary_(sar)']
data['savings_ratio'] = data['elementary_expenses_(sar)'] / data['salary_(sar)']

# Handle infinite or NaN values due to division
data['debt_to_income_ratio'] = data['debt_to_income_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)
data['savings_ratio'] = data['savings_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)

# Drop irrelevant columns
data = data.drop(columns=['name'])

data.describe()
data.head()

,salary_(sar),monthly_debt_(sar),elementary_expenses_(sar),number_of_children,budgeting_rule,state_Single,sex_Male,goal_Savings,employment_status_Student,employment_status_Unemployed,age_group_encoded,debt_to_income_ratio,savings_ratio
0,16409,501.91,829.22,0,70/20/10,True,False,False,False,False,3,0.030587,0.050534
1,2980,457.79,8852.17,2,50/30/20,False,False,False,True,False,0,0.153621,2.970527
3,1679,296.33,12087.75,1,50/30/20,False,False,True,False,True,4,0.176492,7.199375
11,2501,104.38,6806.93,3,50/30/20,False,True,False,False,True,1,0.041735,2.721683
13,1949,540.36,8537.33,2,50/30/20,False,True,True,True,False,1,0.277250,4.380364


Saving Data:

In [500]:
# Save cleaned data to a new Excel file
data.to_excel("Cleaned_Survey_ICS487_Project.xlsx", index=False)

# Or save it as a CSV file
data.to_csv("Cleaned_Survey_ICS487_Project.csv", index=False)